# Import Modules

In [1]:
import tensorflow as tf
import keras
import sys, os
print(tf.__version__)

2025-01-22 16:40:14.350259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
2.10.0


In [18]:
sys.path.append("/Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/modules")
dropbox_path = "/Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/"
october_test_logifold_path = dropbox_path + 'testfolder/logifold_test20241124/'
base_models_path = dropbox_path + 'Base_Models/'
adversarial_examples_path = dropbox_path + 'Adversarial_Examples/'
adversarial_models_path = dropbox_path + 'Adversarial_Models/'

In [19]:
from keras.models import Model, load_model
from keras.datasets import cifar10
import logifoldv1_4 as logifold
from sklearn.model_selection import train_test_split
import numpy as np
from resnet import ResNet
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras import layers
import pickle
import pandas as pd

# Preprocessing Data

x_train = x_tr, x_val = x_v 

y_train = y_tr, y_val = y_v

adv_x_train, adv_x_val, adv_x_test = adversarial examples

training_x, validating_x, testing_x : Polluted data, 1:1, adversaries obtained from ResNet56v1, PGD method with $\epsilon = 0.396$, untargeted. First half is original part, and the other is adversaries.

training_y_long, validating_y_long, testing_y_long : original and adversaries, half and half

filter_y_tr : 80k number of data, consisting of $0$ or $1$ (1:1)

filter_y_val : 20k number of data

In [77]:
(x, y), (x_test, y_test)= cifar10.load_data()
x_tr, x_v, y_tr, y_v = train_test_split(x, y, test_size=0.2, random_state=42)
x_tr = x_tr.reshape(x_tr.shape[0],32,32,3)
x_v = x_v.reshape(x_v.shape[0],32,32,3)
x_v = x_v.astype('float32')
x_tr = x_tr.astype('float32')
x_v /= 255
x_tr /= 255
x_train, x_val, y_train, y_val = x_tr,x_v,y_tr,y_v
x_test = x_test.reshape(x_test.shape[0],32,32,3)
x_test = x_test.astype('float32')
x_test /= 255

training_y_long=np.concatenate([y_train,y_train+10],axis=0)
training_y_long = keras.utils.to_categorical(training_y_long,20)
validating_y_long=np.concatenate([y_v,y_v+10],axis=0)
validating_y_long = keras.utils.to_categorical(validating_y_long,20)
testing_y_long=np.concatenate([y_test,y_test+10],axis=0)
testing_y_long = keras.utils.to_categorical(testing_y_long,20)

filter_y_tr = np.zeros((40000,1),dtype='uint8')
filter_y_tr=np.concatenate([filter_y_tr,filter_y_tr+1],axis=0)
filter_y_tr = keras.utils.to_categorical(filter_y_tr,2)
filter_y_val = np.zeros((10000,1),dtype='uint8')
filter_y_val=np.concatenate([filter_y_val,filter_y_val+1],axis=0)
filter_y_val = keras.utils.to_categorical(filter_y_val,2)
                                          
adv_x_tr = np.load(adversarial_examples_path + "pgd_0.376_x_untarget.npy")
adv_x_val = np.load(adversarial_examples_path + "pgd_0.376_x_val_untarget.npy")
adv_x_test = np.load(adversarial_examples_path+"pgd_0.376_x_test_untarget.npy")
print('adversarial dataset has been loaded.')
training_x = np.concatenate([x_tr,adv_x_tr],axis=0)
validating_x = np.concatenate([x_v,adv_x_val],axis=0)
testing_x = np.concatenate([x_test,adv_x_test],axis=0)


adversarial dataset has been loaded.


# Loading Logifold

In [24]:
lgfd = logifold.Logifold(20,
                        name = "test20241124",
                        x_tr = training_x,
                        y_tr = training_y_long,
                        x_v = validating_x,
                        y_v = validating_y_long, 
                        path = october_test_logifold_path,
                        storyFile='story')
lgfd.load()

In [30]:
print("Logifold 'lgfd' has following models...")
for key in lgfd.keys():
    target = lgfd.charts[key]['target']
    if target == [[n] for n in range(10)]:
        target = 'original 10 classes'
    elif target == [[n] for n in range(10,20)]:
        target = 'Adversarial 10 classes'
    elif len(target) == 20:
        target = 'all 20 classes'
    else:
        target = 'two targets, original or adversarial'
    print(
        f" key = {key}\n targets = {target}\n Description = {lgfd.charts[key]['description']}\n"
    )

Logifold 'lgfd' has following models...
 key = (0, 2, 0)
 targets = Adversarial 10 classes
 Description = specialized from resnet.

 key = (0, 0, 0, 0)
 targets = all 20 classes
 Description = Model trained on (1:1 ratio) original + Adversarial examples generated by untargeted PGD method. Full and fine.

 key = (0, 0, 2)
 targets = two targets, original or adversarial
 Description = filter specialized from 10 targets (full and fine)(1:1 ratio, PGD). Both filters have 99% of accuracy on (polluted) validation dataset  

 key = (0, 0, 0)
 targets = original 10 classes
 Description = resnet56v1

 key = (0, 0, 1)
 targets = two targets, original or adversarial
 Description = filter specialized from 20 targets (full and fine)(1:1 ratio, PGD) 

 key = (0, 1, 0)
 targets = Adversarial 10 classes
 Description = Model trained on Adversarial examples generated by untargeted PGD method



## Prediction and Evaluation

adv model (key : 0,1,0) on original testing dataset

0.783

adv model (key : 0,1,0) on adv testing dataset

0.8868

In [78]:
model = load_model(base_models_path+'CIFAR10models/n_9_v1_cifar10.keras')
prediction1 = model.predict(x_val)
val_accuracy = np.sum(np.argmax(prediction1,axis=-1) == np.argmax(keras.utils.to_categorical(y_v,10),axis=-1))/10000
prediction2 = model.predict(x_test)
test_accuracy = np.sum(np.argmax(prediction2,axis=-1) == np.argmax(keras.utils.to_categorical(y_test,10),axis=-1))/10000
prediction3 = model.predict(adv_x_val)
adv_val_accuracy = np.sum(np.argmax(prediction3,axis=-1) == np.argmax(keras.utils.to_categorical(y_v,10),axis=-1))/10000
prediction4 = model.predict(adv_x_test)
adv_test_accuracy = np.sum(np.argmax(prediction4,axis=-1) == np.argmax(keras.utils.to_categorical(y_test,10),axis=-1))/10000
print(f' ResNet56v1 \n Validation accuracy is {val_accuracy} and test accuracy is {test_accuracy}.\n Accuracy on Adversarial, validation and test, examples are {adv_val_accuracy} and {adv_test_accuracy}, respectively.')

313/313 [==============================] - 27s 85ms/step
 ResNet56v1 
 Validation accuracy is 0.9288 and test accuracy is 0.8785.
 Accuracy on Adversarial, validation and test, examples are 0.0107 and 0.0097, respectively.


In [79]:
keys = [(0,0,0,0)]
EvalOutputFile = lgfd.path+f'evaluations/val_{keys}_00.csv'
data_frame = pd.read_csv(EvalOutputFile)
print('Full and Fine model, on validation dataset')
data_frame

Full and Fine model, on validation dataset


,Unnamed: 0,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0,0.000000,0.9159,0.915900,20000
1,0.500000,0.9159,0.915900,20000
2,0.731059,0.9159,0.915900,20000
3,0.880797,0.9159,0.915900,20000
4,0.952574,0.9159,0.953875,18103
5,0.982014,0.9159,0.983195,14579
6,0.993307,0.9159,0.994349,10086
7,0.997527,0.9159,0.997927,7235
8,0.999089,0.9159,0.999222,6426
9,0.999665,0.9159,0.999809,5240


In [103]:

keys = [(0,0,0,0)]
prediction = lgfd.predict(testing_x,keys = keys , y = testing_y_long,verbose = 1,
                            useHistory = f'evaluations/val_{keys}')
prediction[-4][0][0]


model_000_000_000_000 is thinking...
Vote counting...
Current target of models is: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
Result of the best 1 models in the prescribed list of certainty levels:


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.8269,0.826900,20000
0.500000,0.8269,0.826900,20000
0.731059,0.8269,0.826900,20000
0.880797,0.8269,0.826900,20000
0.952574,0.8269,0.898858,16640
0.982014,0.8269,0.955097,12115
0.993307,0.8269,0.979645,7615
0.997527,0.8269,0.992076,5048
0.999089,0.8269,0.995638,4356
0.999665,0.8269,0.997796,3176


Results using expected accuracy from past history:


{'Accuracy': array([0.8269])}

,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.8269,0.826900,20000
0.500000,0.8269,0.826900,20000
0.731059,0.8269,0.826900,20000
0.880797,0.8269,0.826900,20000
0.952574,0.8269,0.898858,16640
0.982014,0.8269,0.955097,12115
0.993307,0.8269,0.979645,7615
0.997527,0.8269,0.992076,5048
0.999089,0.8269,0.995638,4356
0.999665,0.8269,0.997796,3176


In [88]:
keys = [(0,0,0),(0,1,0),(0,0,1)]
EvalOutputFile = lgfd.path+f'evaluations/val_{keys}_02.csv'
data_frame = pd.read_csv(EvalOutputFile)
print('''On Validation dataset\n[Target Tree]:
                    Filter
          (Specialized from 20 targets)
                       
        Original                 Adversarial
                            (Trained on Adversarial)
      ''')
data_frame

On Validation dataset
[Target Tree]:
                    Filter
          (Specialized from 20 targets)
                       
        Original                 Adversarial
                            (Trained on Adversarial)
      


,Unnamed: 0,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0,0.000000,0.95885,0.958850,20000
1,0.500000,0.95885,0.958850,20000
2,0.731059,0.95885,0.958850,20000
3,0.880797,0.95885,0.958850,20000
4,0.952574,0.95885,0.968724,19568
5,0.982014,0.95885,0.982983,18570
6,0.993307,0.95885,0.988172,17923
7,0.997527,0.95885,0.992035,16572
8,0.999089,0.95885,0.992740,15289
9,0.999665,0.95885,0.992386,13396


In [104]:

keys = [(0,0,0),(0,1,0),(0,0,1)]
prediction = lgfd.predict(testing_x,keys = keys , y = testing_y_long,verbose = 1,
                            useHistory = f'evaluations/val_{keys}')


model_000_000_001 is thinking...
Vote counting...
Current target of models is: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
Result of the best 1 models in the prescribed list of certainty levels:


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.09965,0.09965,20000
0.500000,0.09965,0.00000,0
0.731059,0.09965,0.00000,0
0.880797,0.09965,0.00000,0
0.952574,0.09965,0.00000,0
0.982014,0.09965,0.00000,0
0.993307,0.09965,0.00000,0
0.997527,0.09965,0.00000,0
0.999089,0.09965,0.00000,0
0.999665,0.09965,0.00000,0


Results using expected accuracy from past history:


{'Accuracy': array([0.09965])}

model_000_000_000 is thinking...
Vote counting...
Change in expected acc of prediction using history:


...

Changes compared to the last page (for predictions without using history):


,Old acc,New acc,Number of data
0,0.488900,0.878250,820.0
1,0.488900,0.898540,1578.0
2,0.488900,0.915805,765.0
3,0.488900,0.932974,1324.0
4,0.488900,0.953379,1101.0
5,0.488900,0.962930,1097.0
6,0.488900,0.975771,237.0
7,0.488900,0.977752,2987.0
8,0.870578,0.915805,530.0
9,0.895408,0.932974,912.0


Results using expected accuracy from past history:


{'Accuracy': array([0.09965, 0.4889 , 0.87825])}

,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.09965,0.09965,20000
0.500000,0.09965,0.00000,0
0.731059,0.09965,0.00000,0
0.880797,0.09965,0.00000,0
0.952574,0.09965,0.00000,0
0.982014,0.09965,0.00000,0
0.993307,0.09965,0.00000,0
0.997527,0.09965,0.00000,0
0.999089,0.09965,0.00000,0
0.999665,0.09965,0.00000,0


In [90]:
keys = [(0,0,0),(0,1,0),(0,0,2)]
EvalOutputFile = lgfd.path+f'evaluations/val_{keys}_02.csv'
data_frame = pd.read_csv(EvalOutputFile)
print('''On Validation dataset\n[Target Tree]:
                    Filter
          (Specialized from 10 targets)
                       
        Original                 Adversarial
                            (Trained on Adversarial)
      ''')
data_frame

On Validation dataset
[Target Tree]:
                    Filter
          (Specialized from 10 targets)
                       
        Original                 Adversarial
                            (Trained on Adversarial)
      


,Unnamed: 0,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0,0.000000,0.95825,0.958250,20000
1,0.500000,0.95825,0.958250,20000
2,0.731059,0.95825,0.958250,20000
3,0.880797,0.95825,0.958250,20000
4,0.952574,0.95825,0.968210,19566
5,0.982014,0.95825,0.982602,18565
6,0.993307,0.95825,0.987833,17917
7,0.997527,0.95825,0.991730,16565
8,0.999089,0.95825,0.992475,15282
9,0.999665,0.95825,0.992084,13391


In [105]:

keys = [(0,0,0),(0,1,0),(0,0,2)]
prediction = lgfd.predict(testing_x,keys = keys , y = testing_y_long,verbose = 1,
                            useHistory = f'evaluations/val_{keys}')
prediction[-4][0][0]


model_000_000_002 is thinking...
Vote counting...
Current target of models is: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
Result of the best 1 models in the prescribed list of certainty levels:


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.09965,0.09965,20000
0.500000,0.09965,0.00000,0
0.731059,0.09965,0.00000,0
0.880797,0.09965,0.00000,0
0.952574,0.09965,0.00000,0
0.982014,0.09965,0.00000,0
0.993307,0.09965,0.00000,0
0.997527,0.09965,0.00000,0
0.999089,0.09965,0.00000,0
0.999665,0.09965,0.00000,0


Results using expected accuracy from past history:


{'Accuracy': array([0.09965])}

model_000_000_000 is thinking...
Vote counting...
Change in expected acc of prediction using history:


...

Changes compared to the last page (for predictions without using history):


,Old acc,New acc,Number of data
0,0.488250,0.877550,822.0
1,0.488250,0.897956,1584.0
2,0.488250,0.915539,766.0
3,0.488250,0.932699,1325.0
4,0.488250,0.952868,1104.0
5,0.488250,0.962507,1098.0
6,0.488250,0.975262,237.0
7,0.488250,0.977216,2989.0
8,0.870670,0.915539,528.0
9,0.895465,0.932699,908.0


Results using expected accuracy from past history:


{'Accuracy': array([0.09965, 0.48825, 0.87755])}

,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.09965,0.09965,20000
0.500000,0.09965,0.00000,0
0.731059,0.09965,0.00000,0
0.880797,0.09965,0.00000,0
0.952574,0.09965,0.00000,0
0.982014,0.09965,0.00000,0
0.993307,0.09965,0.00000,0
0.997527,0.09965,0.00000,0
0.999089,0.09965,0.00000,0
0.999665,0.09965,0.00000,0


In [91]:
keys = [(0,0,0),(0,2,0),(0,0,2)]
EvalOutputFile = lgfd.path+f'evaluations/val_{keys}_02.csv'
data_frame = pd.read_csv(EvalOutputFile)
print('''On Validation dataset\n[Target Tree]:
                    Filter
          (Specialized from 10 targets)
                       
        Original                 Adversarial
                            (Specialized from 10 targets)
      ''')
data_frame

On Validation dataset
[Target Tree]:
                    Filter
          (Specialized from 10 targets)
                       
        Original                 Adversarial
                            (Specialized from 10 targets)
      


,Unnamed: 0,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0,0.000000,0.9322,0.932200,20000
1,0.500000,0.9322,0.932200,20000
2,0.731059,0.9322,0.932200,20000
3,0.880797,0.9322,0.932200,20000
4,0.952574,0.9322,0.969910,18179
5,0.982014,0.9322,0.981093,17031
6,0.993307,0.9322,0.986946,15551
7,0.997527,0.9322,0.990153,13100
8,0.999089,0.9322,0.990830,12105
9,0.999665,0.9322,0.990289,10607


In [106]:

keys = [(0,0,0),(0,2,0),(0,0,2)]
prediction = lgfd.predict(testing_x,keys = keys , y = testing_y_long,verbose = 1,
                            useHistory = f'evaluations/val_{keys}')


model_000_000_002 is thinking...
Vote counting...
Current target of models is: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
Result of the best 1 models in the prescribed list of certainty levels:


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.09965,0.09965,20000
0.500000,0.09965,0.00000,0
0.731059,0.09965,0.00000,0
0.880797,0.09965,0.00000,0
0.952574,0.09965,0.00000,0
0.982014,0.09965,0.00000,0
0.993307,0.09965,0.00000,0
0.997527,0.09965,0.00000,0
0.999089,0.09965,0.00000,0
0.999665,0.09965,0.00000,0


Results using expected accuracy from past history:


{'Accuracy': array([0.09965])}

model_000_000_000 is thinking...
Vote counting...
Change in expected acc of prediction using history:


...

Changes compared to the last page (for predictions without using history):


,Old acc,New acc,Number of data
0,0.488250,0.822050,3012.0
1,0.488250,0.882446,1577.0
2,0.488250,0.898838,1556.0
3,0.488250,0.919412,1920.0
4,0.488250,0.939423,469.0
5,0.488250,0.951890,456.0
6,0.488250,0.971520,935.0
7,0.870670,0.898838,528.0
8,0.895465,0.919412,908.0
9,0.927538,0.939423,614.0


Results using expected accuracy from past history:


{'Accuracy': array([0.09965, 0.48825, 0.82205])}

,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.09965,0.09965,20000
0.500000,0.09965,0.00000,0
0.731059,0.09965,0.00000,0
0.880797,0.09965,0.00000,0
0.952574,0.09965,0.00000,0
0.982014,0.09965,0.00000,0
0.993307,0.09965,0.00000,0
0.997527,0.09965,0.00000,0
0.999089,0.09965,0.00000,0
0.999665,0.09965,0.00000,0


Remark. Try Filter(specialized from 20 targets), original, adversarial(specialized from 20 targets).

# Turn full and fine model into an expert on adversaries

In [108]:

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 5, 10, 15, 18 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 18:
        lr *= 0.5e-3
    elif epoch > 15:
        lr *= 1e-3
    elif epoch > 10:
        lr *= 1e-2
    elif epoch > 5:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


lgfd.turnSpecialist(key = (0,0,0,0), newKey=(0,3,0),description='Specialized from full and fine',
                    newTarget = [[n] for n in range(10,20)], lr_schedule = lr_schedule, epochs = 20, verbose = 1)

Learning rate:  0.001
Epoch 1/20
1250/1250 [==============================] - ETA: 0s - loss: 0.4405 - accuracy: 0.8974
Epoch 1: val_accuracy improved from -inf to 0.82750, saving model to /Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/testfolder/logifold_test20241124/model_000_003_000.keras
1250/1250 [==============================] - 371s 294ms/step - loss: 0.4405 - accuracy: 0.8974 - val_loss: 0.6996 - val_accuracy: 0.8275 - lr: 0.0010
Learning rate:  0.001
Epoch 2/20
1250/1250 [==============================] - ETA: 0s - loss: 0.3390 - accuracy: 0.9306
Epoch 2: val_accuracy did not improve from 0.82750
1250/1250 [==============================] - 416s 333ms/step - loss: 0.3390 - accuracy: 0.9306 - val_loss: 0.7171 - val_accuracy: 0.8235 - lr: 0.0010
Learning rate:  0.001
Epoch 3/20
1250/1250 [==============================] - ETA: 0s - loss: 0.3223 - accuracy: 0.9387
Epoch 3: val_accuracy improved f

# Turn full and fine model to expert on original

In [112]:
lgfd.turnSpecialist(key = (0,0,0,0), newKey=(1,0,0),description='Specialized from full and fine',
                    newTarget = [[n] for n in range(10)], lr_schedule = lr_schedule, epochs = 20, verbose = 1)

Learning rate:  0.001
Epoch 1/20
1250/1250 [==============================] - ETA: 0s - loss: 0.5346 - accuracy: 0.8608
Epoch 1: val_accuracy improved from -inf to 0.81670, saving model to /Users/inkeejung/Library/CloudStorage/Dropbox-BOSTONUNIVERSITY/Inkee Jung/Inkee Jung’s files/Interpretability/AdvLogifold/computer/testfolder/logifold_test20241124/model_001_000_000.keras
1250/1250 [==============================] - 364s 289ms/step - loss: 0.5346 - accuracy: 0.8608 - val_loss: 0.6840 - val_accuracy: 0.8167 - lr: 0.0010
Learning rate:  0.001
Epoch 2/20
1250/1250 [==============================] - ETA: 0s - loss: 0.4132 - accuracy: 0.9035
Epoch 2: val_accuracy did not improve from 0.81670
1250/1250 [==============================] - 406s 325ms/step - loss: 0.4132 - accuracy: 0.9035 - val_loss: 0.7196 - val_accuracy: 0.8108 - lr: 0.0010
Learning rate:  0.001
Epoch 3/20
1250/1250 [==============================] - ETA: 0s - loss: 0.3828 - accuracy: 0.9187
Epoch 3: val_accuracy improved f

In [110]:
print('''[Target Tree]:
                                      Filter
                      (Specialized from full and fine)
                       
        Original                                            Adversarial
                                                (Specialized from full and fine)
      ''')

[Target Tree]:
                                      Filter
                      (Specialized from full and fine)
                       
        Original                                            Adversarial
                                                (Specialized from full and fine)
      


In [109]:
keys = [(0,0,0),(0,3,0),(0,0,1)]

prediction1 = lgfd.predict(validating_x,keys = keys , y = validating_y_long,verbose = 1,
                          predOutputFile = f'predictions/val_{keys}',
                          evalOutputFile = f'evaluations/val_{keys}')

prediction2 = lgfd.predict(testing_x,keys = keys , y = testing_y_long,verbose = 1,
                          useHistory = f'evaluations/val_{keys}')

On Validation dataset
[Target Tree]:
                    Filter
          (Specialized from 10 targets)
                       
        Original                 Adversarial
                            (Trained on Adversarial)
      
model_000_000_001 is thinking...
Vote counting...
Current target of models is: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
Result of the best 1 models in the prescribed list of certainty levels:


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.0969,0.0969,20000
0.500000,0.0969,0.0000,0
0.731059,0.0969,0.0000,0
0.880797,0.0969,0.0000,0
0.952574,0.0969,0.0000,0
0.982014,0.0969,0.0000,0
0.993307,0.0969,0.0000,0
0.997527,0.0969,0.0000,0
0.999089,0.0969,0.0000,0
0.999665,0.0969,0.0000,0


model_000_000_000 is thinking...
Vote counting...
Current target of models is: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
Result of the best 2 models in the prescribed list of certainty levels:


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.51265,0.512650,20000
0.500000,0.51265,0.919513,10101
0.731059,0.51265,0.919513,10101
0.880797,0.51265,0.919513,10101
0.952574,0.51265,0.919513,10101
0.982014,0.51265,0.919513,10101
0.993307,0.51265,0.938191,9675
0.997527,0.51265,0.963632,8909
0.999089,0.51265,0.972299,8375
0.999665,0.51265,0.981163,7273


Changes compared to the last page (for predictions without using history):


...

,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.90225,0.902250,20000
0.500000,0.90225,0.902250,20000
0.731059,0.90225,0.902250,20000
0.880797,0.90225,0.902250,20000
0.952574,0.90225,0.909788,19698
0.982014,0.90225,0.922045,18793
0.993307,0.90225,0.938823,17245
0.997527,0.90225,0.957806,15097
0.999089,0.90225,0.967415,13933
0.999665,0.90225,0.979325,11850


Changes compared to the last page (for predictions without using history):


,Old acc,New acc,Number of data
0,0.488900,0.902250,302.0
1,0.488900,0.909788,905.0
2,0.488900,0.922045,1018.0
3,0.488900,0.938823,1236.0
4,0.488900,0.957806,549.0
5,0.488900,0.967415,937.0
6,0.488900,0.981470,4962.0
7,0.870578,0.922045,530.0
8,0.895408,0.938823,912.0
9,0.927737,0.957806,615.0


Results using expected accuracy from past history:


{'Accuracy': array([0.09965, 0.4889 , 0.90225])}

In [111]:
print('''[Target Tree]:
                                        Filter
                            (Specialized from full and fine)
                       
        Original                                            Adversarial
(Specialized from full and fine)                        (Specialized from full and fine)
      ''')

[Target Tree]:
                                        Filter
                            (Specialized from full and fine)
                       
        Original                                            Adversarial
(Specialized from full and fine)                        (Specialized from full and fine)
      


In [113]:
keys = [(1,0,0),(0,3,0),(0,0,1)]

prediction1 = lgfd.predict(validating_x,keys = keys , y = validating_y_long,verbose = 1,
                          predOutputFile = f'predictions/val_{keys}',
                          evalOutputFile = f'evaluations/val_{keys}')

prediction2 = lgfd.predict(testing_x,keys = keys , y = testing_y_long,verbose = 1,
                          useHistory = f'evaluations/val_{keys}')

model_000_000_001 is thinking...
Vote counting...
Current target of models is: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
Result of the best 1 models in the prescribed list of certainty levels:


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.0969,0.0969,20000
0.500000,0.0969,0.0000,0
0.731059,0.0969,0.0000,0
0.880797,0.0969,0.0000,0
0.952574,0.0969,0.0000,0
0.982014,0.0969,0.0000,0
0.993307,0.0969,0.0000,0
0.997527,0.0969,0.0000,0
0.999089,0.0969,0.0000,0
0.999665,0.0969,0.0000,0


model_000_003_000 is thinking...
Vote counting...
Current target of models is: (10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
Result of the best 2 models in the prescribed list of certainty levels:


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.51465,0.514650,20000
0.500000,0.51465,0.941509,9899
0.731059,0.51465,0.941509,9899
0.880797,0.51465,0.941509,9899
0.952574,0.51465,0.957196,9602
0.982014,0.51465,0.984566,8747
0.993307,0.51465,0.994347,7783
0.997527,0.51465,0.998769,6499
0.999089,0.51465,0.999154,5912
0.999665,0.51465,1.000000,4996


Changes compared to the last page (for predictions without using history):


...

,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.9156,0.915600,20000
0.500000,0.9156,0.915600,20000
0.731059,0.9156,0.915600,20000
0.880797,0.9156,0.915600,20000
0.952574,0.9156,0.945687,18688
0.982014,0.9156,0.976397,16396
0.993307,0.9156,0.986443,13867
0.997527,0.9156,0.989554,11296
0.999089,0.9156,0.989122,9744
0.999665,0.9156,0.987448,6931


Changes compared to the last page (for predictions without using history):


,Old acc,New acc,Number of data
0,0.513000,0.915600,1010.0
1,0.513000,0.945687,1387.0
2,0.513000,0.976397,1511.0
3,0.513000,0.986443,1335.0
4,0.513000,0.989554,4848.0
5,0.934504,0.915600,302.0
6,0.950973,0.945687,905.0
7,0.981728,0.976397,1018.0
8,0.992842,0.986443,1236.0
9,0.998139,0.989554,549.0


Results using expected accuracy from past history:


{'Accuracy': array([0.09965, 0.513  , 0.9156 ])}

In [115]:
print('''[Target Tree]:
                        Filter                      +       Full and Fine
                (Specialized from full and fine)
                       
        Original                                            Adversarial
(Specialized from full and fine)                        (Specialized from full and fine)
      ''')

[Target Tree]:
                        Filter                      +       Full and Fine
                (Specialized from full and fine)
                       
        Original                                            Adversarial
(Specialized from full and fine)                        (Specialized from full and fine)
      


In [114]:
keys = [(1,0,0),(0,3,0),(0,0,1),(0,0,0,0)]

prediction1 = lgfd.predict(validating_x,keys = keys , y = validating_y_long,verbose = 1,
                          predOutputFile = f'predictions/val_{keys}',
                          evalOutputFile = f'evaluations/val_{keys}')

prediction2 = lgfd.predict(testing_x,keys = keys , y = testing_y_long,verbose = 1,
                          useHistory = f'evaluations/val_{keys}')

model_000_000_001 is thinking...
model_000_000_000_000 is thinking...
Vote counting...
Current target of models is: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
Result of the best 2 models in the prescribed list of certainty levels:


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.8329,0.832900,20000
0.500000,0.8329,0.832900,20000
0.731059,0.8329,0.832900,20000
0.880797,0.8329,0.832900,20000
0.952574,0.8329,0.902383,16616
0.982014,0.8329,0.958221,12231
0.993307,0.8329,0.981496,7674
0.997527,0.8329,0.991784,4990
0.999089,0.8329,0.995627,4345
0.999665,0.8329,0.998444,3213


model_000_003_000 is thinking...
Vote counting...
Current target of models is: (10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
Result of the best 3 models in the prescribed list of certainty levels:


,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.86370,0.863700,20000
0.500000,0.86370,0.863700,20000
0.731059,0.86370,0.863700,20000
0.880797,0.86370,0.863700,20000
0.952574,0.86010,0.903290,17961
0.982014,0.87125,0.959488,14440
0.993307,0.87065,0.982857,10150
0.997527,0.86780,0.993934,7253
0.999089,0.86650,0.996612,6494
0.999665,0.86555,0.999084,5459


Changes compared to the last page (for predictions without using history):


...

,acc by refined vote,acc by refined vote restricted on confident domain,size of certain part
0.000000,0.89285,0.892850,20000
0.500000,0.89285,0.892850,20000
0.731059,0.89285,0.892850,20000
0.880797,0.89285,0.892850,20000
0.952574,0.89440,0.907893,19434
0.982014,0.90410,0.954729,17318
0.993307,0.90235,0.978100,14612
0.997527,0.89885,0.986871,11882
0.999089,0.89750,0.987862,10298
0.999665,0.89580,0.987309,7407


Changes compared to the last page (for predictions without using history):


,Old acc,New acc,Number of data
0,0.857550,0.892850,566.0
1,0.857550,0.907893,475.0
2,0.857550,0.954729,379.0
3,0.857550,0.978100,217.0
4,0.857550,0.986871,100.0
5,0.857550,0.987862,277.0
6,0.895808,0.907893,1641.0
7,0.895808,0.954729,481.0
8,0.895808,0.978100,366.0
9,0.895808,0.986871,222.0


Results using expected accuracy from past history:


{'Accuracy': array([0.8269 , 0.86785, 0.90805])}